# __Multi-Shell Multi-Tissue CSD (MSMT-CSD) modelling analysis notebook__
#### __Last updated on:__ 27/02/2020
#### __Author:__ Rakshit Dadarwal

### __Requirements:__
#### 1. A bash kernel for IPython (https://github.com/takluyver/bash_kernel)
#### 2. Single-shell DWI dataset (single bvalue)
#### 3. MRtrix3 (https://github.com/MRtrix3/mrtrix3)

### __This script includes:__
#### 1. Preprocessing and multi-tissue CSD modelling

### __Define DWI data path__

In [1]:
#--------------------------------------------------
# Pre-processed DWI data and brain mask 
#--------------------------------------------------
#data_path=/home/rdadarwal/rdadarwal/PRIME_DE_data/dpz/
#dwi_file=eddy_corrected.nii.gz
#brainmask_file=brain_mask.nii.gz
#bval=dMRI.bvals
#bvec=eddy_corrected.eddy_rotated_bvecs
data_path = "/home/erjun/githubEZ/dHCP/sub-CC00060XX03/ses-12501/dwi"
dwi_file = 'sub-CC00060XX03_ses-12501_desc-preproc_dwi.nii.gz'
brainmask_file = 'sub-CC00060XX03_ses-12501_desc-preproc_space-dwi_brainmask.nii.gz'
bval = 'sub-CC00060XX03_ses-12501_desc-preproc_dwi.bval'
bvec = 'sub-CC00060XX03_ses-12501_desc-preproc_dwi.bvec'

In [7]:
# Create directory
mkdir ${data_path}/MS3T-CSD
src_path=${data_path}/MS3T-CSD/

SyntaxError: invalid syntax (<ipython-input-7-4355b5bfd7d4>, line 2)

### __Pre-processing__

In [ ]:
# Convert DWI data from NIfTI into mif (MRtrix Image Format) 
mrconvert ${data_path}/$dwi_file ${src_path}/dwi.mif -fslgrad ${data_path}/$bvec ${data_path}/$bval

# Brain mask
mrconvert ${data_path}/$brainmask_file ${src_path}/dwi_mask.mif

# Bias field correction
dwibiascorrect ants ${src_path}/dwi.mif ${src_path}/dwi_unbiased.mif

### __Post-processing__

In [ ]:
# tissue response function estimation
dwi2response dhollander ${src_path}/dwi_unbiased.mif ${src_path}/response_wm.txt \
${src_path}/response_gm.txt ${src_path}/response_csf.txt

# Fibre Orientation Distribution estimation (multi-tissue CSD modelling)
dwi2fod msmt_csd ${src_path}/dwi_unbiased.mif ${src_path}/response_wm.txt ${src_path}/wmfod.mif ${src_path}/response_gm.txt \
${src_path}/gm.mif ${src_path}/response_csf.txt ${src_path}/csf.mif -mask ${src_path}/dwi_mask.mif

# multi-tissue joint bias field and intensity normalization
mtnormalise ${src_path}/wmfod.mif ${src_path}/wmfod_norm.mif ${src_path}/gm.mif ${src_path}/gm_norm.mif \
${src_path}/csf.mif ${src_path}/csf_norm.mif -mask ${src_path}/dwi_mask.mif

### __Visualisation__

In [ ]:
# Compute an FOD-based DEC map from your WM FOD image
fod2dec ${src_path}/wmfod_norm.mif ${src_path}/decfod.mif -mask ${src_path}/dwi_mask.mif
# Overlay the WM FIDs on the FOD-based DEC map
mrview ${src_path}/decfod.mif -odf.load_sh ${src_path}/wmfod_norm.mif

### __Tractography__

In [ ]:
# Whole-brain tractography
tckgen ${src_path}/wmfod_norm.mif ${src_path}/tracks.tck -seed_image ${src_path}/dwi_mask.mif -select 100000 -cutoff 0.07
# Visualize resulting tractogram
mrview ${src_path}/decfod.mif -tractography.load ${src_path}/tracks.tck -tractography.lighting true